### DMD Ingredient->Product Lookup

Get a list of all VMPs (Virtual Medicinal Products) and AMPs (Actual Medicinal Products) from a given list of ingredients (VTMs = Virtual Therapeutic Moieties)

In [1]:
# set name of codelist for exporting file
codelist_name="covid_meds"


# import or paste list of vtms (Virtual Therapeutic Moieties / Ingredients) by name
names = ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Tocilizumab', 'Sarilumab']




from ebmdatalab import bq
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)



## Select products from DMD containing chosen ingredients

In [2]:
# convert list to tuple for use in SQL query
names_tuple = tuple(names)
if len(names_tuple)==1:
    # remove comma if only one item
    names_tuple = str(names_tuple).replace(",","")

sql = f'''
SELECT "vmp" AS type, vmp.id, bnf_code, vmp.nm, ing.nm AS ingredient, ddd.ddd
FROM dmd.vmp
INNER JOIN dmd.vpi AS vpi ON vmp.id=vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing AND ing.nm IN {names_tuple}
LEFT JOIN dmd.ddd on vmp.id=ddd.vpid

ORDER BY type, nm  '''

meds = bq.cached_read(sql, csv_path=os.path.join('..','data',f'meds_{codelist_name}.csv'))

meds.count()

C:\Users\hcurtis\Anaconda3\envs\antibiotics-rct\lib\site-packages\pandas_gbq\gbq.py:547: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  df = rows_iter.to_dataframe(


type          16
id            16
bnf_code       9
nm            16
ingredient    16
ddd            9
dtype: int64

#### Check if any ingredients were not found

In [3]:
ings = list(meds.ingredient.drop_duplicates())
names.sort()
ings.sort()
print("Selected ingredients:", names)
print("Ingredients found:", ings)

missing_ings = [i for i in names if i not in ings]
print("Ingredients NOT found:", missing_ings)

Selected ingredients: ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Sarilumab', 'Tocilizumab']
Ingredients found: ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Sarilumab', 'Tocilizumab']
Ingredients NOT found: []


In [4]:
# check how many products have DDDs
print(meds[["ddd", "id"]].count())

ddd     9
id     16
dtype: int64


In [5]:
meds

,type,id,bnf_code,nm,ingredient,ddd
0,vmp,39657711000001106,None,Casirivimab 120mg/ml solution for infusion 11.1ml vials,Casirivimab,NaN
1,vmp,40035711000001105,None,Casirivimab 120mg/ml solution for infusion 2.5ml vials,Casirivimab,NaN
2,vmp,39658211000001100,None,Imdevimab 120mg/ml solution for infusion 11.1ml vials,Imdevimab,NaN
3,vmp,40035811000001102,None,Imdevimab 120mg/ml solution for infusion 2.5ml vials,Imdevimab,NaN
4,vmp,38377211000001108,None,Remdesivir 100mg powder for solution for infusion vials,Remdesivir,NaN
5,vmp,38377311000001100,None,Remdesivir 100mg/20ml solution for infusion vials,Remdesivir,NaN
6,vmp,38377411000001107,None,Remdesivir 150mg powder for solution for infusion vials,Remdesivir,NaN
7,vmp,34742511000001107,1001030ADAAACAC,Sarilumab 150mg/1.14ml solution for injection pre-filled disposable devices,Sarilumab,14.3
8,vmp,34742611000001106,1001030ADAAAAAA,Sarilumab 150mg/1.14ml solution for injection pre-filled syringes,Sarilumab,14.3
9,vmp,34742711000001102,1001030ADAAADAD,Sarilumab 200mg/1.14ml solution for injection pre-filled disposable devices,Sarilumab,14.3
